# cliper retrival

In [6]:
import os
import re
import win32clipboard
from win32con import CF_HDROP, CF_UNICODETEXT

def normalize_path(path):
    """
    将路径中的反斜杠和多余的正斜杠规范化为单一的正斜杠。
    例如:
        "C:\\Users\\Admin\\file.txt" -> "C:/Users/Admin/file.txt"
        "C://Users//Admin//file.txt" -> "C:/Users/Admin/file.txt"
    """
    # 将所有反斜杠替换为正斜杠
    path = path.replace('\\', '/')
    # 使用正则表达式替换多个连续的正斜杠为单一正斜杠
    path = re.sub(r'/+', '/', path)
    return path

def get_clipboard_files_info():
    """
    处理剪贴板内容，如果是文件/文件夹则输出路径，
    如果是字符串则判断是否为存在的路径并相应输出。
    输出的路径使用统一的正斜杠作为分隔符。
    """
    norm_files=[]
    try:
        # 打开剪贴板
        win32clipboard.OpenClipboard()
        
        # 处理文件/文件夹
        if win32clipboard.IsClipboardFormatAvailable(CF_HDROP):
            # 获取剪贴板中的文件列表
            files = win32clipboard.GetClipboardData(CF_HDROP)
            print("剪贴板包含以下文件/文件夹路径：")
            for file_path in files:
                # 规范化路径分隔符
                norm_path = normalize_path(file_path)
                print(f"路径: {norm_path}")
                norm_files.append(norm_path)
        
        # 处理字符串
        elif win32clipboard.IsClipboardFormatAvailable(CF_UNICODETEXT):
            # 获取剪贴板中的文本
            text = win32clipboard.GetClipboardData(CF_UNICODETEXT).strip()
            if not text:
                print("剪贴板中的文本为空。")
            else:
                # 规范化路径分隔符
                normalized_text = normalize_path(text)
                # 判断规范化后的路径是否存在
                if os.path.exists(normalized_text):
                    print(f"剪贴板中的字符串是存在的路径：{normalized_text}")
                    norm_files.append(normalized_text)
                else:
                    print(f"剪贴板中的字符串不是路径：{text}")
        
        else:
            print("剪贴板中没有文件、文件夹或文本。")
    
    except Exception as e:
        print(f"处理剪贴板时发生错误: {e}")
    
    finally:
        # 关闭剪贴板
        win32clipboard.CloseClipboard()
        return norm_files


In [8]:
# 调用函数
norm_files=get_clipboard_files_info()

剪贴板包含以下文件/文件夹路径：
路径: C:/Users/Administrator/Downloads/paddleocr/.paddleocr


In [9]:
norm_files

['C:/Users/Administrator/Downloads/paddleocr/.paddleocr']

In [ ]:
C:\Users\Administrator\Downloads\bootstrap-5.3.3-dist.zip
C:/Users/Administrator/Downloads/bootstrap-5.3.3-dist.zip
C://Users//Administrator//Downloads//bootstrap-5.3.3-dist.zip

# main

In [25]:
import os
import zipfile
import re
import win32clipboard
from win32con import CF_HDROP, CF_UNICODETEXT
from shutil import copyfile
from PIL import Image
import datetime
import sys


class FileHyder:
    MARKER = b'FHDR'  # 用于标识加密数据的起始位置

    def __init__(self):
        pass

    def normalize_path(self, path):
        """
        将路径中的反斜杠和多余的正斜杠规范化为单一的正斜杠。
        例如:
            "C:\\Users\\Admin\\file.txt" -> "C:/Users/Admin/file.txt"
            "C://Users//Admin//file.txt" -> "C:/Users/Admin/file.txt"
        """
        path = path.replace('\\', '/')
        path = re.sub(r'/+', '/', path)
        return path

    def get_clipboard_file_path(self):
        """
        从剪贴板中获取文件路径或字符串。
        如果剪贴板中是文件/文件夹，返回路径字符串列表。
        如果剪贴板中是字符串，返回字符串。
        如果剪贴板为空或不支持的格式，返回 None。
        """
        try:
            win32clipboard.OpenClipboard()
            if win32clipboard.IsClipboardFormatAvailable(CF_HDROP):
                files = win32clipboard.GetClipboardData(CF_HDROP)
                normalized_files = [self.normalize_path(file) for file in files]
                return normalized_files
            elif win32clipboard.IsClipboardFormatAvailable(CF_UNICODETEXT):
                text = win32clipboard.GetClipboardData(CF_UNICODETEXT).strip()
                if not text:
                    return None
                normalized_text = self.normalize_path(text)
                # 判断是否为存在的路径
                if os.path.exists(normalized_text):
                    return normalized_text
                else:
                    return text
            else:
                return None
        except Exception as e:
            print(f"从剪贴板获取数据时发生错误: {e}")
            return None
        finally:
            win32clipboard.CloseClipboard()

    def _create_zip(self, file_path):
        """
        将文件或文件夹压缩成 ZIP 文件。
        返回 ZIP 文件路径。
        """
        zip_path = file_path + '.zip'
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            if os.path.isdir(file_path):
                for root, dirs, files in os.walk(file_path):
                    for file in files:
                        abs_path = os.path.join(root, file)
                        rel_path = os.path.relpath(abs_path, os.path.join(file_path, os.pardir))
                        zipf.write(abs_path, rel_path)
            else:
                zipf.write(file_path, os.path.basename(file_path))
        return zip_path

    def _encrypt_file(self, input_path, output_path, key):
        """
        使用 XOR 加密算法对文件进行加密。
        """
        key_bytes = key.encode()
        key_len = len(key_bytes)
        with open(input_path, 'rb') as fin, open(output_path, 'wb') as fout:
            byte = fin.read(1)
            index = 0
            while byte:
                fout.write(bytes([byte[0] ^ key_bytes[index % key_len]]))
                byte = fin.read(1)
                index += 1

    def _decrypt_file(self, input_path, output_path, key):
        """
        使用 XOR 解密算法对文件进行解密。
        """
        self._encrypt_file(input_path, output_path, key)  # XOR 解密与加密相同

    def _create_encrypted_zip(self, file_path, key):
        """
        将文件或文件夹压缩并加密，返回加密后的 ZIP 文件路径。
        """
        zip_path = self._create_zip(file_path)
        encrypted_zip_path = zip_path + '.enc'
        self._encrypt_file(zip_path, encrypted_zip_path, key)
        os.remove(zip_path)  # 删除未加密的 ZIP 文件
        return encrypted_zip_path

    def encode(self, file_path=None, img_path=None, key="qwer1234", output_dir="."):
        """
        将文件或文件夹伪装到图像文件中。
        参数:
            file_path (str): 要伪装的文件或文件夹路径。如果为 None，将从剪贴板获取。
            img_path (str): 伪装用的图像文件路径。如果为 None，将从剪贴板获取。
            key (str): 加密密钥，默认为 "qwer1234"。
            output_dir (str): 输出目录，默认为当前目录。
        返回:
            str: 生成的伪装图像文件的绝对路径。
        """
        # 将 output_dir 转换为绝对路径
        output_dir = os.path.abspath(output_dir)

        # 获取文件路径
        if not file_path:
            clipboard_data = self.get_clipboard_file_path()
            if isinstance(clipboard_data, list):
                if len(clipboard_data) == 1:
                    file_path = clipboard_data[0]
                else:
                    print("剪贴板中包含多个文件，请指定要伪装的单个文件或文件夹路径。")
                    return None
            elif isinstance(clipboard_data, str):
                file_path = clipboard_data
            else:
                print("未提供文件路径，且剪贴板中没有有效的文件路径或字符串。")
                return None

        # 获取图像路径
        if not img_path:
            clipboard_data = self.get_clipboard_file_path()
            if isinstance(clipboard_data, list):
                if len(clipboard_data) == 1 and os.path.isfile(clipboard_data[0]):
                    img_path = clipboard_data[0]
                else:
                    print("剪贴板中包含多个文件或非图像文件，请指定单个图像文件路径。")
                    return None
            elif isinstance(clipboard_data, str):
                img_path = clipboard_data
            else:
                print("未提供图像路径，且剪贴板中没有有效的图像路径或字符串。")
                return None

        # 检查图像文件是否存在
        if not os.path.isfile(img_path):
            print(f"图像文件不存在: {img_path}")
            return None

        # 创建 ZIP 并加密
        encrypted_zip_path = self._create_encrypted_zip(file_path, key)

        # 生成带时间戳的输出文件名
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S-%f")[:-3]
        output_img_name = f"{timestamp}.png"
        output_img_path = os.path.join(output_dir, output_img_name)
        output_img_path = os.path.abspath(output_img_path)  # 转换为绝对路径

        # 创建输出目录如果不存在
        os.makedirs(output_dir, exist_ok=True)

        # 复制原始图像到输出路径
        copyfile(img_path, output_img_path)

        # 将标记和加密的 ZIP 文件附加到图像文件
        with open(output_img_path, 'ab') as img, open(encrypted_zip_path, 'rb') as zip_file:
            img.write(self.MARKER)  # 添加标记
            while True:
                chunk = zip_file.read(4096)
                if not chunk:
                    break
                img.write(chunk)

        # 删除加密的 ZIP 文件
        os.remove(encrypted_zip_path)

        print(f"编码后的文件已创建: {output_img_path}")
        return output_img_path

    def decode(self, img_path=None, key="qwer1234", output_dir="."):
        """
        从伪装的图像文件中提取并解压隐藏的文件或文件夹。
        参数:
            img_path (str): 伪装的图像文件路径。如果为 None，将从剪贴板获取。
            key (str): 解密密钥，默认为 "qwer1234"。
            output_dir (str): 输出目录，默认为当前目录。
        返回:
            str: 解压后的文件或文件夹的绝对路径。
        """
        # 将 output_dir 转换为绝对路径
        output_dir = os.path.abspath(output_dir)

        # 获取图像路径
        if not img_path:
            clipboard_data = self.get_clipboard_file_path()
            if isinstance(clipboard_data, list):
                if len(clipboard_data) == 1 and os.path.isfile(clipboard_data[0]):
                    img_path = clipboard_data[0]
                else:
                    print("剪贴板中包含多个文件或非图像文件，请指定单个图像文件路径。")
                    return None
            elif isinstance(clipboard_data, str):
                img_path = clipboard_data
            else:
                print("未提供图像路径，且剪贴板中没有有效的图像路径或字符串。")
                return None

        # 检查图像文件是否存在
        if not os.path.isfile(img_path):
            print(f"图像文件不存在: {img_path}")
            return None

        # 创建输出文件夹
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        folder_name = os.path.join(output_dir, base_name)
        folder_name = os.path.abspath(folder_name)  # 转换为绝对路径
        os.makedirs(folder_name, exist_ok=True)

        # 读取图像文件并提取加密的 ZIP 数据
        with open(img_path, 'rb') as img_file:
            img_data = img_file.read()
            marker_index = img_data.find(self.MARKER)
            if marker_index == -1:
                print("未在图像文件中找到隐藏的加密 ZIP 数据。")
                return None
            encrypted_zip_data = img_data[marker_index + len(self.MARKER):]

        if not encrypted_zip_data:
            print("没有找到加密的 ZIP 数据。")
            return None

        # 将加密的 ZIP 数据写入临时文件
        temp_encrypted_zip_path = os.path.join(folder_name, "hidden.zip.enc")
        temp_encrypted_zip_path = os.path.abspath(temp_encrypted_zip_path)  # 转换为绝对路径
        with open(temp_encrypted_zip_path, 'wb') as temp_zip_file:
            temp_zip_file.write(encrypted_zip_data)

        # 解密 ZIP 文件
        decrypted_zip_path = os.path.join(folder_name, "hidden.zip")
        decrypted_zip_path = os.path.abspath(decrypted_zip_path)  # 转换为绝对路径
        self._decrypt_file(temp_encrypted_zip_path, decrypted_zip_path, key)
        os.remove(temp_encrypted_zip_path)  # 删除加密的临时 ZIP 文件

        # 解压 ZIP 文件
        try:
            with zipfile.ZipFile(decrypted_zip_path, 'r') as zip_ref:
                zip_ref.extractall(folder_name)
            print(f"解码后的文件已提取到: {folder_name}")
            return folder_name
        except zipfile.BadZipFile:
            print("解密后的文件不是有效的 ZIP 文件。可能是密钥错误或文件损坏。")
            return None
        finally:
            # 删除解密的 ZIP 文件
            if os.path.exists(decrypted_zip_path):
                os.remove(decrypted_zip_path)


# ================== 辅助函数 ==================
def get_clipboard_files_info():
    """
    处理剪贴板内容，如果是文件/文件夹则输出路径，
    如果是字符串则判断是否为存在的路径并相应输出。
    输出的路径使用统一的正斜杠作为分隔符。
    """
    def normalize_path(path):
        """
        将路径中的反斜杠和多余的正斜杠规范化为单一的正斜杠。
        """
        path = path.replace('\\', '/')
        path = re.sub(r'/+', '/', path)
        return path

    try:
        # 打开剪贴板
        win32clipboard.OpenClipboard()

        # 处理文件/文件夹
        if win32clipboard.IsClipboardFormatAvailable(CF_HDROP):
            # 获取剪贴板中的文件列表
            files = win32clipboard.GetClipboardData(CF_HDROP)
            normalized_files = [normalize_path(file) for file in files]
            print("剪贴板包含以下文件/文件夹路径：")
            for path in normalized_files:
                print(f"路径: {path}")

        # 处理字符串
        elif win32clipboard.IsClipboardFormatAvailable(CF_UNICODETEXT):
            # 获取剪贴板中的文本
            text = win32clipboard.GetClipboardData(CF_UNICODETEXT).strip()
            if not text:
                print("剪贴板中的文本为空。")
            else:
                # 规范化路径分隔符
                normalized_text = normalize_path(text)
                # 判断规范化后的路径是否存在
                if os.path.exists(normalized_text):
                    print(f"剪贴板中的字符串是存在的路径：{normalized_text}")
                else:
                    print(f"剪贴板中的字符串不是路径：{text}")

        else:
            print("剪贴板中没有文件、文件夹或文本。")

    except Exception as e:
        print(f"处理剪贴板时发生错误: {e}")

    finally:
        # 关闭剪贴板
        win32clipboard.CloseClipboard()



In [27]:

# ================== 使用示例 ==================

hider = FileHyder()

# 示例：编码
# 传入具体的 file_path 和 img_path，或者将其中一个或两个参数设为 None 以使用剪贴板中的路径
encoded_path = hider.encode(
    file_path=None,        # 如果为 None，将从剪贴板获取
    img_path="temp.png",         # 如果为 None，将从剪贴板获取
    key="qwer1234",        # 加密密钥
    output_dir="output"    # 输出目录，默认为当前目录
)
if encoded_path:
    print(f"编码完成，输出文件路径: {encoded_path}")



编码后的文件已创建: d:\AI\fastshot\output\2024-10-14-00-38-04-442.png
编码完成，输出文件路径: d:\AI\fastshot\output\2024-10-14-00-38-04-442.png


In [24]:
# 示例：解码
# 传入具体的 img_path，或者设为 None 以使用剪贴板中的路径
decoded_folder = hider.decode(
    img_path=None,         # 如果为 None，将从剪贴板获取
    key="qwer1234",        # 解密密钥
    output_dir="decoded"   # 输出目录，默认为当前目录
)
if decoded_folder:
    print(f"解码完成，输出文件夹路径: {decoded_folder}")

# 处理剪贴板内容
# get_clipboard_files_info()


解码后的文件已提取到: decoded\2024-10-14-00-35-32-734
解码完成，输出文件夹路径: decoded\2024-10-14-00-35-32-734


In [28]:

# ================== 使用示例 ==================



hider = FileHyder()

# 示例：编码
# 传入具体的 file_path 和 img_path，或者将其中一个或两个参数设为 None 以使用剪贴板中的路径
encoded_path = hider.encode(
    file_path=None,        # 如果为 None，将从剪贴板获取
    img_path="temp.png",         # 如果为 None，将从剪贴板获取
    key="qwer1234",        # 加密密钥
    output_dir="output"    # 输出目录，默认为当前目录
)



print(f"编码完成，输出文件路径: {encoded_path}")





编码后的文件已创建: d:\AI\fastshot\output\2024-10-14-00-38-16-236.png
编码完成，输出文件路径: d:\AI\fastshot\output\2024-10-14-00-38-16-236.png


In [29]:

# 示例：解码
# 传入具体的 img_path，或者设为 None 以使用剪贴板中的路径
decoded_folder = hider.decode(
    img_path=None,         # 如果为 None，将从剪贴板获取
    key="qwer1234",        # 解密密钥
    output_dir="decoded"   # 输出目录，默认为当前目录
)
print(f"解码完成，输出文件夹路径: {decoded_folder}")

# 处理剪贴板内容
# get_clipboard_files_info()


解码后的文件已提取到: d:\AI\fastshot\decoded\2024-10-14-00-38-04-442
解码完成，输出文件夹路径: d:\AI\fastshot\decoded\2024-10-14-00-38-04-442
